In [ ]:
#!pip install psycopg2-binary #for Michelle

In [ ]:
import numpy as np
import pandas as pd
import os, glob
import sqlalchemy
import sqlite3
from sqlite3 import Error
import sklearn as sk
from sklearn.linear_model import LogisticRegression
from sklearn import svm, metrics
#from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import math
import gzip
import csv
import matplotlib.pyplot as plt
import psycopg2

# Connecting to mimic postgres database

In [ ]:
from sqlalchemy import create_engine

# Postgres username, password, database name
# @team - you'll need to change this according to what your Postgres info is under the Properties tab of your server in PgAdmin

# Michelle's info
#POSTGRES_ADDRESS = '127.0.0.1'
#POSTGRES_PORT= '5432'
#POSTGRES_USERNAME= 'postgres'
#POSTGRES_PASSWORD= 'password'
#POSTGRES_DBNAME = 'mimic'
#postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,ipaddress=POSTGRES_ADDRESS,port=POSTGRES_PORT,dbname=POSTGRES_DBNAME))

# Paroma's info 
POSTGRES_ADDRESS = 'localhost'   #127.0.0.1
POSTGRES_PORT= '5432'
POSTGRES_USERNAME= 'MacUser'
POSTGRES_PASSWORD= ''
POSTGRES_DBNAME = 'mimic'

# putting in info
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}?options=-csearch_path%3Dmimiciii'.format(username=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,ipaddress=POSTGRES_ADDRESS,port=POSTGRES_PORT,dbname=POSTGRES_DBNAME))

# Create the connection
cnx = create_engine(postgres_str)


In [ ]:
# path with all un-normalized data -chartevents, lab events, procedure events
#path = "C:/Users/mnigc/Documents/PCM/" # for Michelle
path = "/Users/MacUser/Desktop/Fall_2020_Semester/Classes/Precision_Care_Medicine/Un-normalized_matched_patient_id_Data"

# FOR LABEVENTS
# loading all files from path with labevent name
cohort_files = glob.glob(os.path.join(path, "lab_events_*"))

# initializing data frame
cohort_df = []
# loading all files into data frame
for f in cohort_files:
    df = pd.read_csv(f)
    df['file'] = f.split('/')[-1]
    cohort_df.append(df)
    
# concatenating files into single data frame    
cohort = pd.concat(cohort_df, ignore_index=True)

C:\Users\mnigc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
#  getting subject ids from cohort files
patient_ids = cohort['subject_id']

# get unique patient ids
unique_ids=np.unique(patient_ids)

# clear NaN
unique_ids=unique_ids[~np.isnan(unique_ids)]

# 2594 patients
#print(len(unique_ids))

# for each unique_ids, determine if each patient is alive (0) or dead (1)
# 1403 dead patients and 1191 alive patients
patient_list = [];
for i in unique_ids:
    query = 'SELECT subject_id, expire_flag FROM patients WHERE subject_id = {};'.format(i)
    pat = pd.read_sql_query(query, cnx)
    patient_list.append(pat)
patient = pd.concat(patient_list, ignore_index=True) 
num_patients_dead = patient['expire_flag'].sum()
num_patients_alive = len(unique_ids) - num_patients_dead;
print("Number of dead patients: ", num_patients_dead)
print("Number of alive patients: ", num_patients_alive)


2596

# Feature extraction and cleaning by group

In [ ]:
# find all the item ids - 


In [ ]:
# Albumin
# 50862 (Albumin), 51069 (Albumin, Urine), 51070 (Albumin/Creatinine/Urine)

albumin = pd.read_sql_query('''SELECT * FROM labevents WHERE itemid = 50862 OR itemid = 51069 OR itemid = 51070''',cnx)
#print(albumin) # 151240 rows
albumin1 = albumin[albumin['subject_id'].isin(unique_ids)] 
print(albumin1) # 16230 rows

#alb_list = [];
#for index, value in albumin_itemids.items():
#    query = 'SELECT * FROM labevents WHERE itemid = ({});'.format(albumin_itemids[index])
#    alb = pd.read_sql_query(query, cnx)
#    alb_list.append(alb)
#albumin = pd.concat(alb_list, ignore_index=True)
#albumin1 = albumin[albumin['subject_id'].isin(unique_ids)] 
#print(albumin1)

   itemid         linksto
0    3066     chartevents
1   45403  inputevents_cv
   subject_id  hadm_id           charttime value valuenum warning
0        1046   113191 2168-10-21 23:00:00  None     None    None
Empty DataFrame
Columns: [subject_id, hadm_id, charttime, amount]
Index: []


In [ ]:
# Anion Gap (calculated from Na, K, Cl, HCO3)
# 50868 (Anion Gap)

anion_id = pd.read_sql_query('''SELECT itemid, linksto FROM d_items WHERE label='%Anion%';''',cnx)
anion_itemids = anion_id['itemid']
    
#an_list = [];
#for index, value in anion_itemids.items():
#    query = 'SELECT * FROM labevents WHERE itemid = {};'.format(anion_itemids[index])
#    an = pd.read_sql_query(query, cnx)
#    an_list.append(an)
#anion = pd.concat(an_list, ignore_index=True)
#anion1 = anion[anion['subject_id'].isin(unique_ids)] 
#print(anion1)


aniongap = pd.read_sql_query('''SELECT * FROM labevents WHERE 
itemid = 50868''',cnx)
#print(aniongap) # 769810 rows
aniongap1 = aniongap[aniongap['subject_id'].isin(unique_ids)] 
print(aniongap1) # 75608 rows

In [ ]:
# Bilirubin
# Bilirubin (51464), Bilirubin Crystals (51465)

# Michelle's Code
# find bilirubin ids from d_items 
#bilirubin_id=pd.read_sql_query('''SELECT itemid,label FROM d_labitems WHERE label LIKE '%Bilirubin%' or label LIKE '%Bilirubin' or label LIKE or label ='Bilirubin';''',cnx)
#print(bilirubin_id)

# find albumin entries in linksto tables for cohort
#bilirubin_chart=pd.read_sql_query('''SELECT subject_id, hadm_id, charttime, value, valuenum, warning 
#FROM chartevents
#WHERE itemid IN (3066);''',cnx)
#albumin_input=pd.read_sql_query('''SELECT subject_id, hadm_id, charttime, amount FROM inputevents_cv WHERE itemid =45403''',cnx)

# check to see if these people are in cohort
#albumin_chart=albumin_chart[albumin_chart['subject_id'].isin(unique_ids)]
#albumin_input=albumin_input[albumin_input['subject_id'].isin(unique_ids)]

# Paroma's code
bilirubin = pd.read_sql_query('''SELECT * FROM labevents WHERE 
itemid = 51464 OR 
itemid = 51465''',cnx)
#print(bilirubin) # 99763 rows
bilirubin1 = bilirubin[bilirubin['subject_id'].isin(unique_ids)] 
print(bilirubin1) # 10375 rows


In [ ]:
# Chloride
# 50806 (Chloride, Whole Blood), 50902 (Chloride), 51078 (Chloride, Urine)

chloride = pd.read_sql_query('''SELECT * FROM labevents WHERE 
itemid = 50806 OR 
itemid = 50902 OR 
itemid = 51078''',cnx)
#print(chloride) # 853268 rows
chloride1 = chloride[chloride['subject_id'].isin(unique_ids)] 
print(chloride1) # 81899 rows

In [ ]:
# Creatinine
# 50912 (Creatinine), 51067 (24 hr Creatinine), 51070 (Albumin), 
# 51073 (Amylase/Creatinine Ratio/Urine), 51080 (Creatinine Clearance),
# 51081 (Creatinine, Serum), 51082 (Creatinine, Urine), 51099 (Protein/Creatinine Ratio),
# 51106 (Urine Creatinine)

creatinine = pd.read_sql_query('''SELECT * FROM labevents WHERE 
itemid = 50912 OR 
itemid = 51067 OR 
itemid = 51070 OR
itemid = 51073 OR
itemid = 51080 OR
itemid = 51081 OR
itemid = 51082 OR
itemid = 51099 OR
itemid = 51106''',cnx)
#print(creatinine) # 841484 rows
creatinine1 = creatinine[creatinine['subject_id'].isin(unique_ids)] 
print(creatinine1) # 83938 rows

In [ ]:
# Diastolic BP
# found in MIMIC-III Waveform Database Matched Subset

In [ ]:
# Epinephrine

In [ ]:
# Glasgow Coma Scale

In [ ]:
# Glucose
# 50809 (Glucose), 50931 (Glucose), 51084 (Glucose, Urine), 51478 (Glucose)
# 51529 (Estimated Actual Glucose)

glucose = pd.read_sql_query('''SELECT * FROM labevents WHERE 
itemid = 50809 OR 
itemid = 50931 OR 
itemid = 51084 OR
itemid = 51478 OR
itemid = 51529''',cnx)
#print(glucose) # 1047294 rows
glucose1 = glucose[glucose['subject_id'].isin(unique_ids)] 
print(glucose1) # 109154 rows

In [ ]:
# Heart rate
# found in MIMIC-III Waveform Database Matched Subset

In [ ]:
# Hemoglobin
# 50811 (Hemoglobin), 50852 (% Hemoglobin A1c), 50855 (Absolute Hemoglobin),
# 51212 (Fetal Hemoglobin), 51222 (Hemoglobin), 51223 (Hemoglobin A2),
# 51224 (Hemoglobin C), 51225 (Hemoglobin F), 
# 51285 (Reticulocyte, Cellular Hemoglobin)

hemoglobin = pd.read_sql_query('''SELECT * FROM labevents WHERE 
itemid = 50811 OR 
itemid = 50852 OR 
itemid = 50855 OR
itemid = 51212 OR
itemid = 51222 OR
itemid = 51223 OR
itemid = 51224 OR
itemid = 51225 OR
itemid = 51285''',cnx)
#print(hemoglobin) # 863043 rows
hemoglobin1 = hemoglobin[hemoglobin['subject_id'].isin(unique_ids)] 
print(hemoglobin1) # 85822 rows

In [ ]:
# Intubation

In [ ]:
# Lactate
# 50813 (Lactate), 50954 (Lactate Dehydrogenase (LD))

lactate = pd.read_sql_query('''SELECT * FROM labevents WHERE 
itemid = 50813 OR 
itemid = 50954''',cnx)
#print(lactate) # 294236 rows
lactate1 = lactate[lactate['subject_id'].isin(unique_ids)] 
print(lactate1) # 27199 rows

In [ ]:
# Mean BP
# found in MIMIC-III Waveform Database Matched Subset

In [ ]:
# Mechanical ventilation

In [ ]:
# pH
# 50820 (pH), 51094 (pH), 51491 (pH)

pH = pd.read_sql_query('''SELECT * FROM labevents WHERE 
itemid = 50820 OR 
itemid = 51094 OR 
itemid = 51491''',cnx)
#print(pH) # 647990 rows
pH1 = pH[pH['subject_id'].isin(unique_ids)] 
print(pH1) # 75593 rows

In [ ]:
# Platelet
# 51240 (Large Platelets), 51264 (Platelet Clumps), 51265 (Platelet Count)
# 51266 (Platelet Smear)

platelet = pd.read_sql_query('''SELECT * FROM labevents WHERE 
itemid = 51240 OR 
itemid = 51264 OR 
itemid = 51265 OR 
itemid = 51266''',cnx)
#print(platelet) # 829614 rows
platelet1 = platelet[platelet['subject_id'].isin(unique_ids)] 
print(platelet1) # 82640 rows

In [ ]:
# Potassium
# 50822 (Potassium, Whole Blood), 50971 (Potassium), 
# 51097 (Potassium, Urine)

potassium = pd.read_sql_query('''SELECT * FROM labevents WHERE 
itemid = 50822 OR 
itemid = 50971 OR 
itemid = 51097''',cnx)
#print(potassium) # 1049617 rows
potassium1 = potassium[potassium['subject_id'].isin(unique_ids)] 
print(potassium1) # 107268 rows

In [ ]:
# Respiration rate

In [ ]:
# Sodium
# 50824 (Sodium, Whole Blood), 50983 (Sodium), 51100 (Sodium, Urine)

sodium = pd.read_sql_query('''SELECT * FROM labevents WHERE 
itemid = 50824 OR 
itemid = 50983 OR 
itemid = 51100''',cnx)
#print(sodium) # 905895 rows
sodium1 = sodium[sodium['subject_id'].isin(unique_ids)] 
print(sodium1) # 88594 rows

In [ ]:
# SOFA Score ?? ? ?  - Still need to get feedback from mentors re: inclusion

In [ ]:
# SpO2
# found in MIMIC-III Waveform Database Matched Subset
# also found it in lab events in mimiciii regular database
# 50817 (Oxygen Saturation)
SpO2_id = pd.read_sql_query('''SELECT itemid,label,fluid FROM d_labitems WHERE label LIKE '%Oxygen Saturation%';''',cnx)

spo2 = pd.read_sql_query('''SELECT * FROM labevents WHERE itemid = 50817''',cnx)
#print(spo2) # 173418 rows
spo2_1 = spo2[spo2['subject_id'].isin(unique_ids)] 
print(spo2_1) #23979 rows

In [ ]:
# Systolic BP
# found in MIMIC-III Waveform Database Matched Subset

In [ ]:
# Temperature
# 50825 (Temperature)
Temp_id = pd.read_sql_query('''SELECT itemid,label,fluid FROM d_labitems WHERE label LIKE '%Temp%';''',cnx)

temp = pd.read_sql_query('''SELECT * FROM labevents WHERE itemid = 50825''',cnx)
#print(temp) # 129444 rows
temp1 = temp[temp['subject_id'].isin(unique_ids)] 
print(temp1) # 18162 rows

In [ ]:
# Urine output
Urine_id = pd.read_sql_query('''SELECT itemid,label,fluid FROM d_labitems WHERE label LIKE '%Urine%';''',cnx)

# 42 different Urine related labels
# Need help deciding which ones to extract

In [ ]:
# White blood count
# https://www.cancer.gov/publications/dictionaries/cancer-terms/def/white-blood-cell
# Types of WBCs: granulocytes (neutrophils, eosinophils, and basophils), 
# monocytes, and lymphocytes (T cells and B cells)
# also include CD4, CD8, etc

# 51130 (Absolute CD3 Count), 51131 (Absolute CD4 Count)
# 51132 (Absolute CD8 Count), 51133 (Absolute Lymphocyte Count)
# 51162 (CD16/56 Absolute Count), 51166 (CD19 Absolute Count)
# 51170 (CD20 Absolute Count), 51175 (CD3 Absolute Count)
# 51186 (CD5 Absolute Count), 51199 (Eosinophil Count)
# 51218 (Granulocyte), 51253 (Monocyte Count), 51300 (WBC Count)

WBC_id = pd.read_sql_query('''SELECT itemid,label,fluid FROM d_labitems WHERE label LIKE '%Count%';''',cnx)

# Decided to cut down to the following labels
# 51130 (Absolute CD3 Count), 51131 (Absolute CD4 Count)
# 51132 (Absolute CD8 Count), 51133 (Absolute Lymphocyte Count)
# 51218 (Granulocyte), 51300 (WBC Count)

wbc = pd.read_sql_query('''SELECT * FROM labevents WHERE 
itemid = 51130 OR 
itemid = 51131 OR 
itemid = 51132 OR 
itemid = 51133 OR 
itemid = 51218 OR 
itemid = 51300''',cnx)
#print(wbc) # 35600 rows
wbc1 = wbc[wbc['subject_id'].isin(unique_ids)] 
print(wbc1) # 3212 rows
